In [ ]:
from bs4 import BeautifulSoup             
import os
import re
from nltk.corpus import stopwords
import pandas as pd


import gensim
from gensim import models
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

from gensim.corpora import Dictionary

In [ ]:
stemmer = SnowballStemmer('english')
stops = set(stopwords.words("english"))

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def lyric_to_words( raw_lyric ):
    lyric_text = BeautifulSoup(raw_lyric, features="lxml").get_text() 
    letters_only = re.sub("[^a-zA-Z]", " ", lyric_text) 
    words = letters_only.lower().split()                                               
    meaningful_words = [w for w in words if not w in stops]
    lemmatize_stemming_words = [lemmatize_stemming(w) for w in words] 

    return lemmatize_stemming_words
    # return( " ".join( meaningful_words ))

In [ ]:
data_set = pd.read_csv("tcc_ceds_music.csv", delimiter=",")
data_set = data_set.drop(["dating", "violence", "world/life", "night/time", "family/gospel", "shake the audience", "dating", "violence", "world/life", "sadness", "feelings", "danceability", "loudness",	"acousticness",	"instrumentalness",	"valence",	"energy", "topic", "romantic",	"communication",	"obscene",	"music",	"movement/places",	"light/visual perceptions",	"family/spiritual",	"like/girls", "age"], axis=1)
data_set.head()

In [ ]:
%%time
processed_lyrics = data_set['lyrics'].map(lyric_to_words)

In [ ]:
data_set['processed_lyrics'] = processed_lyrics

In [ ]:
%%time
dictionary = Dictionary(processed_lyrics)

In [ ]:
%%time
dictionary.filter_extremes(no_below=15, no_above=0.25, keep_n=100000)

In [ ]:
%%time
corpus = [dictionary.doc2bow(lyric) for lyric in processed_lyrics]

In [ ]:
%%time
lda_model = gensim.models.LdaMulticore(corpus, num_topics=9, id2word=dictionary, passes=10)

In [ ]:
topics = [sorted(lda_model[c], key=lambda tup: -1*tup[1])[0][0] for c in corpus]
data_set["topics"] = topics

In [ ]:
import pyLDAvis.gensim_models

lda_display = pyLDAvis.gensim_models.prepare(
    lda_model,
    corpus,
    dictionary,
    sort_topics=False,
)
pyLDAvis.display(lda_display)

In [ ]:
data_set.describe(include='all')